In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
tqdm.pandas()

In [3]:
SOURCES = ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 'JiraEcosystem', 'MariaDB', 'Mindville', 'MongoDB', 'Qt', 'RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring']

In [4]:
def add_linked_issues_to_df(df):
    df['issues']=''
    for i in tqdm(range(len(df))):
        df["issues"].iloc[i] = str(sorted(set([df.iloc[i]['issue_id_1'], df.iloc[i]['issue_id_2']])))

In [6]:
def load_data(source):
    #Loading Issues
    filename = '../data/crawl/issues_'+source.lower()+'.csv'
    issue_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';', index_col=['issue_id'])

    #Loading Links
    filename = '../data/crawl/links_'+source.lower()+'.csv'
    link_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';').drop_duplicates()
    
    return issue_df, link_df

In [8]:
def clean_issues(issue_df):
    #Remove issues with empty titles
    issue_df = issue_df[~issue_df['title'].fillna(' ').str.isspace()]
    print(f'After filtering out issues with empty titles, {len(issue_df)} issues remain')
    
    return issue_df

In [9]:
def clean_links(link_df):
    add_linked_issues_to_df(link_df)
    
    # remove links with uncrawled and filtered issues
    link_df = link_df[link_df[['issue_id_1', 'issue_id_2']].isin(issue_df.index.values).all(axis=1)]
    print(f'Left with {len(link_df)} links after removing half-private links')
    
    # cleanup links
    # only allow one linktype per issue-pair
    link_df.drop_duplicates(subset=['name'], keep=False, inplace=True)
    print(f'Left with {len(link_df)} links after removing issue-pairs with multiple links between them')

    # in case the name is the otherway around, like issue-1_issue-2 and issue-2_issue-1
    doublelinks = (link_df.issues.value_counts()>1).rename_axis('doubles').reset_index(name='valid')
    valid_doubles = set(doublelinks[doublelinks['valid']==True]['doubles'])

    for i in tqdm(valid_doubles):
        if len(set(link_df[link_df['issues']==i]['linktype']))>1:
            link_df = link_df[link_df.issues != i]
    print(f'Left with {len(link_df)} links after removing issue-pairs with multiple link types between them')

    #Multiple links complete remove
    link_df.drop_duplicates(subset=['issues'], inplace=True)
    print(f'Left with {len(link_df)} links after removing issue-pairs with multiple entries')

    link_df.reset_index(inplace=True, drop=True)
    
    return link_df

In [10]:
for s in SOURCES:
    print(s.upper())
    issue_df, link_df = load_data(s)
    print(f'Loaded {len(issue_df)} issues and {len(link_df)} links')
    
    issue_df = clean_issues(issue_df)
    
    link_df = clean_links(link_df)
    
    print(f'Cleaned {len(issue_df)} issues and {len(link_df)} links')
    
    link_df.to_csv('../data/crawl/clean_links_'+s.lower()+'.csv', encoding='utf-8', index=True, sep=';')
    print("----------------------------")

APACHE
Loaded 970929 issues and 250890 links
After filtering out issues with empty titles, 970928 issues remain


100%|██████████| 250890/250890 [10:28<00:00, 398.96it/s]


Left with 250437 links after removing half-private links
Left with 246641 links after removing issue-pairs with multiple links between them


/usr/local/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
100%|██████████| 2145/2145 [01:42<00:00, 20.88it/s]


Left with 243295 links after removing issue-pairs with multiple link types between them
Left with 242823 links after removing issue-pairs with multiple entries
Cleaned 970928 issues and 242823 links
----------------------------
HYPERLEDGER
Loaded 27914 issues and 16765 links
After filtering out issues with empty titles, 27914 issues remain


...

Left with 4290 links after removing issue-pairs with multiple link types between them
Left with 4289 links after removing issue-pairs with multiple entries
Cleaned 77835 issues and 4289 links
----------------------------
SPRING
Loaded 69100 issues and 14715 links
After filtering out issues with empty titles, 69100 issues remain


100%|██████████| 14715/14715 [00:07<00:00, 1859.37it/s]


Left with 14615 links after removing half-private links
Left with 14549 links after removing issue-pairs with multiple links between them


100%|██████████| 52/52 [00:00<00:00, 337.81it/s]


Left with 14477 links after removing issue-pairs with multiple link types between them
Left with 14461 links after removing issue-pairs with multiple entries
Cleaned 69100 issues and 14461 links
----------------------------


## Example of what exactly is cleaned

In [20]:
source = 'Apache'
issue_df, link_df = load_data(source)

print(f'Loaded {len(issue_df)} issues and {len(link_df)} links')

Loaded 970929 issues and 250890 links


In [21]:
add_linked_issues_to_df(link_df)

100%|██████████| 250890/250890 [05:49<00:00, 718.45it/s]


In [22]:
# remove links with uncrawled and filtered issues
cl_link_df = link_df[link_df[['issue_id_1', 'issue_id_2']].isin(issue_df.index.values).all(axis=1)]
print(f'Left with {len(cl_link_df)} links after removing half-private links')

Left with 250437 links after removing half-private links


In [23]:
cl_link_df.name.value_counts()

IMPALA-4835_IMPALA-6594      3
BEAM-1251_BEAM-4511          3
IMPALA-4835_IMPALA-6592      3
HAWQ-832_HAWQ-808            3
IMPALA-4835_IMPALA-6587      3
                            ..
KUDU-430_KUDU-1127           1
MESOS-3647_MESOS-3665        1
ARTEMIS-2687_ARTEMIS-2704    1
DATALAB-213_DATALAB-461      1
LUCENE-1465_LUCENE-1542      1
Name: name, Length: 248533, dtype: int64

In [24]:
cl_link_df[cl_link_df['name']=='IMPALA-4835_IMPALA-6594']

,name,linktype,issue_id_1,issue_id_2,issues
209312,IMPALA-4835_IMPALA-6594,Regression,IMPALA-4835,IMPALA-6594,"['IMPALA-4835', 'IMPALA-6594']"
209313,IMPALA-4835_IMPALA-6594,dependent,IMPALA-4835,IMPALA-6594,"['IMPALA-4835', 'IMPALA-6594']"
210015,IMPALA-4835_IMPALA-6594,Subtask,IMPALA-4835,IMPALA-6594,"['IMPALA-4835', 'IMPALA-6594']"


In [25]:
cl_link_df.drop_duplicates(subset=['name'], keep=False, inplace=True)
print(f'Left with {len(cl_link_df)} links after removing issue-pairs with multiple links between them')

Left with 246641 links after removing issue-pairs with multiple links between them


<ipython-input-25-ca4bd122a4fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cl_link_df.drop_duplicates(subset=['name'], keep=False, inplace=True)


In [34]:
# in case the name is the otherway around, like issue-1_issue-2 and issue-2_issue-1
doublelinks = (cl_link_df.issues.value_counts()>1).rename_axis('doubles').reset_index(name='valid')
valid_doubles = set(doublelinks[doublelinks['valid']==True]['doubles'])

In [35]:
valid_doubles

{"['SVN-3401', 'SVN-3818']",
 "['LANG-1124', 'LANG-1237']",
 "['YARN-3587', 'YARN-3598']",
 "['HADOOP-12693', 'HADOOP-16509']",
 "['CASSANDRA-2848', 'CASSANDRA-5683']",
 "['HIVE-16489', 'HIVE-17237']",
 "['SVN-1327', 'SVN-4240']",
 "['SVN-3504', 'SVN-3709']",
 "['HIVE-2935', 'HIVE-3122']",
 "['HUDI-305', 'HUDI-907']",
 "['SVN-1284', 'SVN-3357']",
 "['DRILL-2459', 'DRILL-3216']",
 "['REEF-1117', 'REEF-989']",
 "['FLINK-8592', 'FLINK-9653']",
 "['SVN-1374', 'SVN-1718']",
 "['WHIRR-598', 'WHIRR-673']",
 "['INFRA-13078', 'INFRA-13227']",
 "['HDFS-7826', 'HDFS-7937']",
 "['SPARK-10346', 'SPARK-12235']",
 "['THRIFT-1366', 'THRIFT-1388']",
 "['SVN-2754', 'SVN-2821']",
 "['PHOENIX-3181', 'PHOENIX-3252']",
 "['THRIFT-3301', 'THRIFT-3954']",
 "['PIG-4266', 'PIG-4304']",
 "['THRIFT-487', 'THRIFT-488']",
 "['OFBIZ-7073', 'OFBIZ-7538']",
 "['SVN-1974', 'SVN-3629']",
 "['MSITE-443', 'MSITE-444']",
 "['NIFI-2014', 'NIFI-2024']",
 "['SLING-4575', 'SLING-5698']",
 "['SPARK-27216', 'SPARK-27530']",
 "['

In [36]:
cl_link_df[(cl_link_df['issue_id_1'] == 'SVN-3401') | (cl_link_df['issue_id_2'] == 'SVN-3401')]

,name,linktype,issue_id_1,issue_id_2,issues
183828,LANG-1237_LANG-1124,Reference,LANG-1237,LANG-1124,"['LANG-1124', 'LANG-1237']"
183829,LANG-1124_LANG-1237,Reference,LANG-1124,LANG-1237,"['LANG-1124', 'LANG-1237']"


In [37]:
for i in valid_doubles:
    if len(set(cl_link_df[cl_link_df['issues']==i]['linktype']))>1:
        cl_link_df = cl_link_df[cl_link_df.issues != i]
    else:
        print(i)
print(f'Left with {len(cl_link_df)} links after removing issue-pairs with multiple link types between them')

  0%|          | 6/2145 [00:00<01:42, 20.87it/s]

['LANG-1124', 'LANG-1237']


  1%|          | 19/2145 [00:00<01:18, 27.10it/s]

['WHIRR-598', 'WHIRR-673']
['HDFS-7826', 'HDFS-7937']
['SPARK-10346', 'SPARK-12235']


  1%|          | 25/2145 [00:01<01:27, 24.14it/s]

...

['HIVE-1534', 'HIVE-1621']
['TS-1489', 'TS-1840']
['NIFI-271', 'NIFI-587']


100%|█████████▉| 2135/2145 [01:20<00:00, 28.89it/s]

['MDEPLOY-57', 'MSHARED-36']
['NIFI-1933', 'NIFI-1934']


100%|██████████| 2145/2145 [01:20<00:00, 26.65it/s]

['SPARK-11628', 'SPARK-9685']
['HDFS-14796', 'HDFS-14797']
['NETBEANS-2494', 'NETBEANS-2978']
Left with 243295 links after removing issue-pairs with multiple link types between them


In [38]:
cl_link_df.issues.value_counts()

['HADOOP-12855', 'HADOOP-12946']     2
['CAMEL-9444', 'CAMEL-9573']         2
['MESOS-2299', 'MESOS-2301']         2
['AVRO-1965', 'HIVE-15316']          2
['HADOOP-9008', 'MAPREDUCE-4780']    2
                                    ..
['FLINK-14574', 'FLINK-15216']       1
['SENTRY-1869', 'SENTRY-872']        1
['ISIS-188', 'ISIS-52']              1
['SPARK-20073', 'SPARK-6231']        1
['FOP-1891', 'FOP-2745']             1
Name: issues, Length: 242823, dtype: int64

In [40]:
cl_link_df[(cl_link_df['issue_id_1'] == 'HADOOP-12855') | (cl_link_df['issue_id_2'] == 'HADOOP-12855')]

,name,linktype,issue_id_1,issue_id_2,issues
288833,HADOOP-12946_HADOOP-12855,Reference,HADOOP-12946,HADOOP-12855,"['HADOOP-12855', 'HADOOP-12946']"
288835,HADOOP-12855_HADOOP-12946,Reference,HADOOP-12855,HADOOP-12946,"['HADOOP-12855', 'HADOOP-12946']"
289007,HADOOP-12908_HADOOP-12855,Reference,HADOOP-12908,HADOOP-12855,"['HADOOP-12855', 'HADOOP-12908']"


In [41]:
#Multiple links complete remove
cl_link_df.drop_duplicates(subset=['issues'], inplace=True)
print(f'Left with {len(cl_link_df)} links after removing issue-pairs with the same link type with multiple entries')

cl_link_df.reset_index(inplace=True, drop=True)

Left with 242823 links after removing issue-pairs with the same link type with multiple entries
